**Introduction to PromptGPT**

PromptGPT is managed using Control panel, which in this case is simply a jupyter notebook.

**Control panel**
* Installing LLM package & importing its library (e.g. OpenAI API, Anthropic)
* PromptGPT functions.
* PromptGPTs pre-defined settings.

**Setup**

In [ ]:
!pip install -r requirements.txt
import openai
from prompgpt import *
openai.api_key = os.getenv("OPENAI_API_KEY")

**Settings**

In [ ]:
# SYSTEM SETTTINGS
#prompt = input("Ask a question: ")
#prompt = "Organize and execute a plan to generate 1000 followers in twitter without spending money in 5 steps."
valid_statuses ="Completed, In progress, Error, Need help"
prompt = "Generate 1000 followers with zero budget."
system_instruction = "You are a world-class prompt engineer producing right prompts to LLM in JSON format only to support programming anything. Only one overall objective. Prompts correctly solve anything by mapping multi-step, short, precise, measurable task/sub-task descriptions & role to perform them & necessary resource/memory name & use only status messages: {status_messages}."
model_to_use = 'gpt-4' #'gpt-3.5-turbo'# 
max_length_input = 500
filepath = 'short_memory.txt'
execution_length = 1000
define_task = 'Print only the task name from the json: {}.'
define_role = "Print only the task role from the json: {}."
define_resource = "Print only the task resource from the json: {}."
define_memory = "Print only the task memory from the json: {}."
define_task_system = f"You are a task selector. You only print name of the task. You only pick one task, which is first task not completed yet and in status in progresss."
define_role_system = f"You are a role selector. You only print name of the role of the task. You only pick one role, which is first role not completed yet and in status in progresss." 
define_resource_system = f"You are a resource selector. You only print resource of the task. You only pick one resource, which is first resource not completed yet and in status in progresss." 
define_memory_system = f"You are a task memory selector. You only print memory name of the task. You only pick one memory, which is first memory not completed yet and in status in progresss. In case no memory required: print No memory"
execution_script = f"print only the solution as a json:"
execution_length = 1000
execution_system_instruction =  "You are a world-class {} completing {} correctly. You have available resource: {}. Generate a step-by-step answer." 


**Getting started**

GPT can now be programmed to perform tasks like:
* Planning
* Add information to memory
* Retrieve information from memory
* Execute task in the list

In [ ]:
# PLANNING
x = gpt_program(prompt, system_instruction, model_to_use, max_length_input)
#print(x)

In [ ]:
#MEMORY
save_memory(x, filepath)
#print(get_memory(filepath, ''))
memory = get_memory(filepath, '')
print(memory)

In [ ]:
# TASKING
task = get_task(define_task, define_task_system)
task_role = get_role(define_role, define_role_system)
task_resource = get_resource(define_resource, define_resource_system)
print(task + "/ " + task_role + "/ " + task_resource)

In [ ]:
# EXECUTION
executed_json = execute(execution_script, execution_system_instruction)
print(executed_json)

**Next steps:**
The plan is to make a composable system, where different functions like planning, execution and memory retrieval are used repeated times without needs for manual intervention. The current system can be given an input. Next step will save it memory, retrieve the first task description, role and resources. Then it will use these to execute the plan. These prompts are rough, but seem steerable.

1. Include external resources and a function for them.
2. Improve the memory
3. Improve the control panel pre-defined system and instruction prompts